### Imports

In [1]:
import sys
import os, shutil
import gc
import torch
import copy
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import open3d
import transforms3d
import open3d as o3d
from sklearn.neighbors import NearestNeighbors
import glob
from torch.utils.data import Dataset
from open3d import read_point_cloud
from tqdm import tqdm_notebook
import pandas as pd
from multiprocessing import Pool, cpu_count

In [2]:
%matplotlib notebook

In [3]:
def getint(name):
    return int(name.split('.')[0])

### Setting paths and constants

In [4]:
VOXEL_SZ = 0.2

In [5]:
pcd_folder_path = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/small_map/110k/static/1"

pcd_folder = "_out"
pcd_path = os.path.join(pcd_folder_path, pcd_folder)
pcd_files = sorted(os.listdir(pcd_path), key=getint)

In [6]:
idx = np.random.choice(len(pcd_files))
# idx = 2925
# idx = 1841
# idx = 3062
step = 10
idx, step

(3487, 10)

In [7]:
prev_pcd_file = pcd_files[idx]
next_pcd_file = pcd_files[idx+step]

### My I/O functions

In [8]:
def filter_pcd(old_pcd,
               apply_downsample = False,
               downsample_voxel_size = 0.2,
               
               apply_outlier_removal = False,
               downsample_radius = 1,
               downsample_neighbors = 20,
               
               apply_crop = True,
               crop_min_arr = np.array([-100,-100,-2]),
               crop_max_arr = np.array([100,100,100]),
#                crop_min_arr = np.array([-100,-100,-100]),
#                crop_max_arr = np.array([100,100,2]),
               
               apply_cluster = False,
               cluster_neighbours = 30,
               cluster_labels = 2):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)
    
    if apply_outlier_removal:
        denser_pcd, ind = o3d.geometry.radius_outlier_removal(pcd,
                                                              nb_points = downsample_neighbors,
                                                              radius    = downsample_radius)
        pcd = denser_pcd
    
    if apply_downsample:
        voxel_down_pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size = downsample_voxel_size)
        pcd = voxel_down_pcd
    
    if apply_crop:
        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd

    if apply_cluster:
        few_pts = np.asarray(pcd.points)
        try:
            few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2).fit_transform(few_pts)
        except Exception as e:
            try:
                few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2, eigen_solver='dense').fit_transform(few_pts)
            except Exception as e:
                few_pts_reduced = few_pts
        clf = MeanShift().fit(few_pts_reduced)
        pcd.points = o3d.utility.Vector3dVector(few_pts[clf.labels_ < cluster_labels])
    
    return pcd

def make_2d(pcd):
    new_pcd = copy.deepcopy(pcd)
    new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
    new_pcd.points = o3d.utility.Vector3dVector(new_pts)
    return new_pcd

def read_pcd(pcd_id):
#     prefix = "".join(["0" for _ in range(3 - len(str(pcd_id)))])
    pcd_file = str(pcd_id) + PCD_TYPE
    pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_2d'):    
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], s=1)
        plt.show()
        
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    return transform_mat
  
def draw_registration_result(src, dst, transformation):
    source = copy.deepcopy(src)
    target = copy.deepcopy(dst)
    
    source.paint_uniform_color([1, 0, 0]) # red
    target.paint_uniform_color([0, 0, 1]) # blue
    target.transform(transformation)
#     o3d.visualization.draw_geometries([source, target], width=1280, height=800)
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(source)
    visualizer.add_geometry(target)
    visualizer.show()

In [9]:
prev_pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, prev_pcd_file))
next_pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, next_pcd_file))

### Ground Truth Transformation

In [10]:
df_gt = pd.read_csv(os.path.join(pcd_folder_path, "groundTruth.csv"))

In [11]:
prev_pcd_file,next_pcd_file

('3488.ply', '3498.ply')

In [12]:
prev_gt = df_gt[df_gt['step'] == idx+1].iloc[0]
next_gt = df_gt[df_gt['step'] == idx+1+step].iloc[0]

In [13]:
prev_gt['step'], next_gt['step']

(3488.0, 3498.0)

In [14]:
transformation_lidar2gt = pose2matrix([0,0,0], [0,0,90],[1,1,-1])
transformation_gt2lidar = np.linalg.inv(transformation_lidar2gt)
prev_pcd.transform(transformation_lidar2gt)
next_pcd.transform(transformation_lidar2gt)

geometry::PointCloud with 8885 points.

In [15]:
prev_inv_mat = np.linalg.inv(pose2matrix([prev_gt['location_x'],
                                          prev_gt['location_y'],
                                          prev_gt['location_z']],
                                         [prev_gt['rotation_roll'],
                                          prev_gt['rotation_pitch'],
                                          prev_gt['rotation_yaw']]))
next_mat = pose2matrix([next_gt['location_x'],
                        next_gt['location_y'],
                        next_gt['location_z']],
                       [next_gt['rotation_roll'],
                        next_gt['rotation_pitch'],
                        next_gt['rotation_yaw']])
transformation_gt = np.matmul(prev_inv_mat, next_mat)

In [16]:
draw_registration_result(prev_pcd, next_pcd, transformation_gt)

JVisualizer with 2 geometries

### Open 3d ICP transformation

In [17]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
    radius_normal = voxel_size * 2
    o3d.geometry.estimate_normals(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size):
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

def refine_registration(source, target, voxel_size, trans_init):
    distance_threshold = voxel_size * 0.4
    result = o3d.registration.registration_icp(
                source, target, distance_threshold, trans_init,
                o3d.registration.TransformationEstimationPointToPlane())
    return result

def get_pose(src, dst):
    source = filter_pcd(src)
    target = filter_pcd(dst)
#     source result_icptransform_mat= copy.deepcopy(src)
#     target = copy.deepcopy(dst)

    voxel_size = VOXEL_SZ
    source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source, target, voxel_size)

    result_ransac = execute_global_registration(source_down, target_down,
                                                source_fpfh, target_fpfh,
                                                voxel_size)

    result_icp = refine_registration(source, target, voxel_size, result_ransac.transformation)
    
    
    evaluation = o3d.registration.evaluate_registration(source, target, voxel_size * 5, result_icp.transformation)
    return result_icp.transformation, evaluation

In [18]:
transformation_icp, evaluation_icp = get_pose(prev_pcd, next_pcd)

transformation_icp = np.linalg.inv(transformation_icp)
transformation_icp

array([[ 9.99976537e-01, -3.39276572e-03, -5.95109740e-03,
         5.84933588e+00],
       [ 3.39989192e-03,  9.99993515e-01,  1.18775260e-03,
         7.95201962e-02],
       [ 5.94702904e-03, -1.20795782e-03,  9.99981587e-01,
         1.35747995e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [19]:
evaluation_icp

registration::RegistrationResult with fitness = 0.898507, inlier_rmse = 0.245808, and correspondence_set size of 1806
Access transformation to get result.

In [20]:
draw_registration_result(prev_pcd, next_pcd, transformation_icp)

JVisualizer with 2 geometries

### RPE error

In [23]:
transformation_rpe =  np.matmul(np.linalg.inv(transformation_gt), transformation_icp)

In [24]:
trans_arr, rot_mat, scale_mat, shear_mat = transforms3d.affines.decompose44(transformation_rpe)
rot_list = transforms3d.euler.mat2euler(rot_mat, axes='sxyz')
rot_arr = np.array(rot_list)
np.linalg.norm(trans_arr)+np.linalg.norm(rot_arr)# ,np.linalg.norm(trans_arr), np.linalg.norm(rot_arr)

(0.2973728617385134, 0.2630236635591657, 0.034349198179347736)